In [1]:
import torch.nn as nn
import torch
import torch.nn.functional as F
import math,copy
import warnings
warnings.simplefilter("ignore")
print(torch.__version__)

1.11.0+cpu


In [4]:
import numpy as np
import pandas as pd

In [2]:
embed = nn.Embedding(21, 5)

In [10]:
embed(torch.tensor([1,2]))

tensor([[ 1.1130,  0.2931, -0.9396, -0.2664,  0.0778],
        [-1.1823, -0.7777, -0.6811,  0.4352, -0.4074]],
       grad_fn=<EmbeddingBackward0>)

In [5]:
with open('seq_enz_data_split_500.npy', 'rb') as f:
    a = np.load(f)

In [6]:
with open('seq_non_enz_data_split_500.npy', 'rb') as f:
    b = np.load(f)

In [7]:
data = np.array(list(a)+list(b))

In [12]:
dict_ = list(pd.Series(a[0]).unique())

In [13]:
encoded_data = []
for i in data:
    tem = []
    for j in range(data.shape[1]):
        try:
            tem.append( dict_.index(i[j]) )
        except:
            tem.append(0)
    encoded_data.append(tem)

In [96]:
target = list(np.zeros((600,1),dtype=int))+list(np.ones((599,1),dtype=int))  #+list(np.zeros_like(b,dtype=int))

In [97]:
target_t = []
for i in target:
    if i[0] == 0:
        target_t.append([1,0])
    else:
        target_t.append([0,1])

In [102]:
target_t[600]

[0, 1]

In [86]:
np.array([0,1])

array([0, 2])

In [14]:
embed_data = []
for i in encoded_data:
    embed_data.append(embed(torch.tensor(i)))

In [22]:
print(embed_data[0][0])

tensor([ 0.2214, -0.4445,  0.6064,  1.6632, -0.2181],
       grad_fn=<SelectBackward0>)


In [32]:
embed_data[0].detach().numpy()

array([[ 0.22136997, -0.44450822,  0.6064224 ,  1.6631844 , -0.21809869],
       [ 1.1129605 ,  0.29309446, -0.939614  , -0.26642385,  0.0778375 ],
       [-1.1823006 , -0.7777363 , -0.6810685 ,  0.4352491 , -0.40738904],
       ...,
       [-0.03570509,  0.60774314,  0.0155361 ,  0.03611943,  0.12822247],
       [-0.03570509,  0.60774314,  0.0155361 ,  0.03611943,  0.12822247],
       [-0.03570509,  0.60774314,  0.0155361 ,  0.03611943,  0.12822247]],
      dtype=float32)

In [126]:
embed_data_batch = []
for X in embed_data:
    embed_data_batch.append( np.array(np.array(X.detach().numpy()).transpose()))

In [127]:
np.array(embed_data_batch).shape

(1199, 5, 500)

In [158]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.conv1 = nn.Conv1d(5,200,3)
#         self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv1d(200, 100, 3)
        self.fc1 = nn.Linear(100 * 496 , 20)
        self.fc2 = nn.Linear(20, 2)
        self.soft = nn.Softmax(dim=1)
    def forward(self, x):
        x = self.conv1(x)
        x = nn.functional.relu(self.conv2(x))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        x = self.soft(x)
        return x

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (conv1): Conv1d(5, 200, kernel_size=(3,), stride=(1,))
  (conv2): Conv1d(200, 100, kernel_size=(3,), stride=(1,))
  (fc1): Linear(in_features=49600, out_features=20, bias=True)
  (fc2): Linear(in_features=20, out_features=2, bias=True)
  (soft): Softmax(dim=1)
)


In [155]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [151]:
# embed_data[0].transpose((1, 0))
np.array([[0,1],
         [2,3],
         [5,6]]).transpose()

array([[0, 2, 5],
       [1, 3, 6]])

In [156]:
def train(embed_data,target_ten, model, loss_fn, optimizer):
    n = 0
    model.train()
    loss_t = 0
    X,y = embed_data,target_ten
#     for X, y in zip(embed_data,target_ten):
#         X, y = torch.tensor(np.array([np.array(X.detach().numpy()).transpose()])).to(device), torch.tensor(np.array([y])).to(device)
    X, y = torch.tensor(X).to(device), torch.tensor(np.array(y)).to(device)
    y = y.float()
    X = X.float()
#         print(X.shape)
    pred = model(X)
    if n == 0:
#         print(pred.shape,pred)
#         print(y.shape,y)
        n += 1
    loss = loss_fn(pred, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    loss = loss.item()
#         print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
    loss_t += loss
    print(loss_t)

In [141]:
epochs = 20
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(embed_data_batch,target_t, model, loss_fn, optimizer)
    #test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
torch.Size([1199, 2]) tensor([[0.4927, 0.5073],
        [0.4872, 0.5128],
        [0.5087, 0.4913],
        ...,
        [0.5002, 0.4998],
        [0.4891, 0.5109],
        [0.4956, 0.5044]], grad_fn=<SoftmaxBackward0>)
torch.Size([1199, 2]) tensor([[1., 0.],
        [1., 0.],
        [1., 0.],
        ...,
        [0., 1.],
        [0., 1.],
        [0., 1.]])
0.6931829452514648
Epoch 2
-------------------------------
torch.Size([1199, 2]) tensor([[0.7625, 0.2375],
        [0.7854, 0.2146],
        [0.7637, 0.2363],
        ...,
        [0.6215, 0.3785],
        [0.6196, 0.3804],
        [0.6517, 0.3483]], grad_fn=<SoftmaxBackward0>)
torch.Size([1199, 2]) tensor([[1., 0.],
        [1., 0.],
        [1., 0.],
        ...,
        [0., 1.],
        [0., 1.],
        [0., 1.]])
0.6965106725692749
Epoch 3
-------------------------------
torch.Size([1199, 2]) tensor([[0.2562, 0.7438],
        [0.2411, 0.7589],
        [0.2661, 0.7339],
        ...,
 

0.6292114853858948
Done!


In [143]:
def test(embed_data,target_ten, model, loss_fn, optimizer):
    n = 0
    model.eval()
    loss_t = 0
    X,y = embed_data,target_ten
#     for X, y in zip(embed_data,target_ten):
#         X, y = torch.tensor(np.array([np.array(X.detach().numpy()).transpose()])).to(device), torch.tensor(np.array([y])).to(device)
    X, y = torch.tensor(X).to(device), torch.tensor(np.array(y)).to(device)
    y = y.float()
    X = X.float()
#         print(X.shape)
    pred = model(X)
    if n == 0:
        print(pred.shape,pred)
        print(y.shape,y)
        n += 1
    loss = loss_fn(pred, y)
#     optimizer.zero_grad()
#     loss.backward()
#     optimizer.step()
    loss = loss.item()
#         print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
    loss_t += loss
    print(loss_t)

In [145]:
epochs = 1
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    test(embed_data_batch,target_t, model, loss_fn, optimizer)
    #test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
torch.Size([1199, 2]) tensor([[0.6480, 0.3520],
        [0.5847, 0.4153],
        [0.7042, 0.2958],
        ...,
        [0.0826, 0.9174],
        [0.0742, 0.9258],
        [0.1667, 0.8333]], grad_fn=<SoftmaxBackward0>)
torch.Size([1199, 2]) tensor([[1., 0.],
        [1., 0.],
        [1., 0.],
        ...,
        [0., 1.],
        [0., 1.],
        [0., 1.]])
0.6218894720077515
Done!


In [153]:
epochs = 20
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(embed_data_batch,target_t, model, loss_fn, optimizer)
    #test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
torch.Size([1199, 2]) tensor([[0.4472, 0.5528],
        [0.4582, 0.5418],
        [0.4454, 0.5546],
        ...,
        [0.4407, 0.5593],
        [0.4348, 0.5652],
        [0.4557, 0.5443]], grad_fn=<SoftmaxBackward0>)
torch.Size([1199, 2]) tensor([[1., 0.],
        [1., 0.],
        [1., 0.],
        ...,
        [0., 1.],
        [0., 1.],
        [0., 1.]])
0.6943832039833069
Epoch 2
-------------------------------
torch.Size([1199, 2]) tensor([[0.9937, 0.0063],
        [0.9960, 0.0040],
        [0.9931, 0.0069],
        ...,
        [0.9707, 0.0293],
        [0.9701, 0.0299],
        [0.9788, 0.0212]], grad_fn=<SoftmaxBackward0>)
torch.Size([1199, 2]) tensor([[1., 0.],
        [1., 0.],
        [1., 0.],
        ...,
        [0., 1.],
        [0., 1.],
        [0., 1.]])
0.803155779838562
Epoch 3
-------------------------------
torch.Size([1199, 2]) tensor([[0.7789, 0.2211],
        [0.8286, 0.1714],
        [0.7756, 0.2244],
        ...,
  

0.676830530166626
Done!


In [157]:
epochs = 20
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(embed_data_batch,target_t, model, loss_fn, optimizer)
    #test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
0.6929490566253662
Epoch 2
-------------------------------
0.6680653691291809
Epoch 3
-------------------------------
0.8136221766471863
Epoch 4
-------------------------------
0.8136786818504333
Epoch 5
-------------------------------
0.8136786818504333
Epoch 6
-------------------------------
0.8136786818504333
Epoch 7
-------------------------------
0.8136786818504333
Epoch 8
-------------------------------
0.8136786818504333
Epoch 9
-------------------------------
0.8136786818504333
Epoch 10
-------------------------------
0.8136786818504333
Epoch 11
-------------------------------
0.8136786818504333
Epoch 12
-------------------------------
0.8136786818504333
Epoch 13
-------------------------------
0.8136786818504333
Epoch 14
-------------------------------
0.8136786818504333
Epoch 15
-------------------------------
0.8136786818504333
Epoch 16
-------------------------------
0.8136786818504333
Epoch 17
-------------------------------
0.813678